In [1]:
import pandas as pd

In [ ]:
# lab = pd.read_csv('lab.csv')
# chart = pd.read_csv('chart.csv')
# diagnoses = pd.read_csv('diagnoses.csv')
# procedures = pd.read_csv('procedures.csv')
# inputs = pd.read_csv('inputs.csv')
# outputs = pd.read_csv('outputs.csv')
# noteEvents = pd.read_csv("NOTEEVENTS.csv")
# patients = pd.read_csv("PATIENTS.csv")
# admissions = pd.read_csv("ADMISSIONS.csv")
# prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
noteEvents = pd.read_csv("NOTEEVENTS.csv")
patients = pd.read_csv("PATIENTS.csv")
admissions = pd.read_csv("ADMISSIONS.csv")
labEvents = pd.read_csv('LABEVENTS.csv')
dict_labItems = pd.read_csv('D_LABITEMS.csv')
chartEvents = pd.read_csv('CHARTEVENTS.csv')
dict_Items = pd.read_csv('D_ITEMS.csv')
diagnosesList = pd.read_csv('DIAGNOSES_ICD.csv')
dict_diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv')
drg_diag = pd.read_csv('DRGCODES.csv')
procedureEvents = pd.read_csv('PROCEDURES_ICD.csv')
dict_procedures = pd.read_csv('D_ICD_PROCEDURES.csv')
prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
inputEvents_cv = pd.read_csv('INPUTEVENTS_CV.csv')
inputEvents_mv = pd.read_csv('INPUTEVENTS_MV.csv')
outputEvents = pd.read_csv('OUTPUTEVENTS.csv')

master = pd.read_csv('master.csv')

In [2]:
prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
patients = pd.read_csv("PATIENTS.csv")
admissions = pd.read_csv("ADMISSIONS.csv")
master = pd.read_csv('master.csv')
diagnosesList = pd.read_csv('DIAGNOSES_ICD.csv')
dict_diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv')
dict_labItems = pd.read_csv('D_LABITEMS.csv')
labEvents = pd.read_csv('LABEVENTS.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
master

,SUBJECT_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,delirium,AGE
0,23,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,71
1,25,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,ACUTE CORONARY SYNDROME,0,59
3,31,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,STATUS EPILEPTICUS,1,72
4,32,2170-04-04 08:00:00,2170-04-23 12:45:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,TRACHEAL STENOSIS/SDA,0,46
6,36,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,70
7,37,2183-08-21 16:48:00,2183-08-26 18:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,CONGESTIVE HEART FAILURE,1,69
8,38,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,76
9,41,2132-12-31 10:30:00,2133-01-27 15:45:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,RIGHT BRAIN STEM LESION/SDA,0,56
10,357,2198-08-02 04:49:00,2198-10-26 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,SEIZURE,1,62
11,359,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,R/O MYOCARDIAL INFARCTION,0,49


In [85]:
#Calculate age + Mark gender

pat_admsn = patients.merge(admissions, how = 'outer', left_on = 'SUBJECT_ID', right_on = 'SUBJECT_ID')
pat_admsn_age = pat_admsn[['SUBJECT_ID', 'GENDER', 'DOB', 'ADMITTIME']]
pat_admsn_age['DOB'] = pat_admsn_age['DOB'].str.slice(stop = 4)
pat_admsn_age['ADMITTIME'] = pat_admsn_age['ADMITTIME'].str.slice(stop = 4)
pat_admsn_age['AGE'] = 0
for i in pat_admsn_age.index: 
    admit_time = int(pat_admsn_age['ADMITTIME'][i])
    dob = int(pat_admsn_age['DOB'][i])
    age = admit_time - dob
    pat_admsn_age['AGE'][i] = age
    
    gen = pat_admsn_age['GENDER'][i]
    if gen == 'M': 
        pat_admsn_age['GENDER'][i] = 1
    elif gen == 'F':
        pat_admsn_age['GENDER'][i] = 0
    else: 
        pat_admsn_age['GENDER'][i] = None
 
    
pat_admsn_age = pat_admsn_age[['SUBJECT_ID', 'AGE']]
pat_admsn_age = pat_admsn_age.drop_duplicates(subset = 'SUBJECT_ID')
 
master = master.merge(pat_admsn_age, how = 'left', left_on = 'SUBJECT_ID', right_on = 'SUBJECT_ID')

keep = master[master['AGE'] < 150]
keep = keep[keep["AGE"] > 10]
master = keep
master = master.drop_duplicates(subset = 'SUBJECT_ID')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [74]:
##DRUGS

def has_string(text, string):
    if string.lower() in text.lower():
        return 1
    return 0

benzodiazepines = ['chlordiazepoxide', 'clorazepate', 'diazepam', 'oxazepam', 'temazepam', 'clonazepam', 
                   'alprazolam', 'lorazepam', 'triazolam']
opioids = ['Buprenorphine', 'Butorphanol', 'Codeine', 'Fentanyl', 'Hydrocodone', 'Hydromorphone', 
           'Meperidine', 'Methadone', 'Morphine', 'Oxycodone', 'Oxymorphone', 'Tapentadol', 'Tramadol']
anticholinergic = ['Amitriptyline', 'Atropine', 'Benztropine', 'Chlorpheniramine', 'Chlorpromazine', 'Clomipramine', 
                   'Clozapine', 'Cyclobenzaprine', 'Cyproheptadine', 'Desipramine', 'Dexchlorpheniramine', 
                   'Dicyclomine', 'Diphenhydramine', 'Doxepin', 'Fesoterodine', 'Hydroxyzine', 'Hyoscyamine', 
                   'Imipramine', 'Meclizine', 'Nortriptyline', 'Olanzapine', 'Orphenadrine', 'Oxybutynin', 
                   'Paroxetine', 'Perphenazine', 'Prochlorperazine', 'Promethazine', 'Protriptyline', 
                   'Pseudoephedrine', 'Scopolamine', 'Thioridazine', 'Tolterodine', 'Trifluoperazine', 'Trimipramine',
                   'Alprazolam', 'Amantadine', 'Baclofen', 'Carisoprodol', 'Cetirizine', 'Cimetidine', 'Clorazepate', 
                   'Codeine', 'Colchicine', 'Digoxin', 'Diphenoxylate', 'Fluphenazine', 'Furosemide', 
                   'Hydrochlorothiazide', 'Loperamide', 'Loratadine', 'Maprotiline', 'Nifedipine', 'Ranitidine', 
                   'Thiothixene', 'Tizanidine']
steroids = ['Fludrocortisone', 'dexamethasone', 'Methylprednisolone', 'triamcinolone', 'prednisolone', 'prednisone', 
            'ethamethasoneb', 'cortisone', 'hydrocortisone', 'bethamethasone']
deep_sedation = ['propofol', 'ketamine', 'etomidate', 'barbiturate']



drugs = prescriptions
drugs = drugs[['SUBJECT_ID', 'DRUG']]


drugs['benzodiazepine'] = 0
drugs['opioid'] = 0
drugs['anticholinergic'] = 0
drugs['steroid'] = 0
drugs['deep sedation'] = 0

for i in drugs.index:
    drug_name = drugs['DRUG'][i]
    for b in benzodiazepines: 
        if has_string(drug_name, b):
            drugs['benzodiazepine'][i] = 1
    for o in opioids:
        if has_string(drug_name, o):
            drugs['opioid'][i] = 1
    for a in anticholinergic:
        if has_string(drug_name, a): 
            drugs['anticholinergic'][i] = 1
    for s in steroids:
        if has_string(drug_name, s):
            drugs['steroid'][i] = 1
    for d in deep_sedation:
        if has_string(drug_name, d):
            drugs['deep sedation'][i] = 1

drugs = drugs.drop(columns=['DRUG'])

master1 = master[['SUBJECT_ID', 'delirium']]

master_drugs = pd.merge(master1, drugs, how = 'left', on = 'SUBJECT_ID')
master_drugs = master_drugs[['SUBJECT_ID', 'benzodiazepine', 'opioid', 'anticholinergic', 'steroid', 'deep sedation']]
master_drugs = master_drugs.dropna()
master_drugs = master_drugs.groupby(['SUBJECT_ID']).agg(max)
#master_drugs.to_csv('masterDrugs.csv', index = False)  
master_drugs

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,benzodiazepine,opioid,anticholinergic,steroid,deep sedation
SUBJECT_ID,,,,,
11,0.0,1.0,1.0,1.0,0.0
12,1.0,1.0,1.0,0.0,1.0
17,0.0,1.0,1.0,0.0,1.0
21,0.0,1.0,1.0,0.0,0.0
23,1.0,1.0,1.0,1.0,1.0
25,0.0,0.0,1.0,0.0,0.0
31,1.0,1.0,1.0,1.0,0.0
36,1.0,1.0,1.0,0.0,1.0
37,1.0,1.0,1.0,0.0,0.0


In [75]:
###Alcohol/Drug Abuse
#search for "abuse" in dict and use those labels

def has_string(text, string):
    if string.lower() in text.lower():
        return 1
    return 0

dict_diagnoses['alcohol/drug abuse'] = dict_diagnoses.apply(lambda row: has_string(row['LONG_TITLE'], 'abuse'), axis = 1)

abuse = dict_diagnoses[dict_diagnoses['alcohol/drug abuse'] == 1]
abuse = abuse.iloc[0:36]

abuse_diag = pd.merge(diagnosesList, abuse, how = 'left', on = 'ICD9_CODE')


master_abuse = pd.merge(master, abuse_diag, how = 'left', on = 'SUBJECT_ID')
master_abuse = master_abuse[['SUBJECT_ID', 'alcohol/drug abuse']]
master_abuse = master_abuse.groupby(['SUBJECT_ID']).agg(max)
master_abuse = master_abuse.fillna(0)
master_abuse

,alcohol/drug abuse
SUBJECT_ID,
11,0.0
12,0.0
17,0.0
21,0.0
23,0.0
25,0.0
31,0.0
32,0.0
36,0.0


In [76]:
###DIAGNOSES/DISORDERS

##metabolic disorders

metabolic = dict_diagnoses.copy()
metabolic['metabolic disorder'] = 0
for i in metabolic.index:
    if metabolic['ICD9_CODE'][i].startswith('27'):
        metabolic['metabolic disorder'][i] = 1 

metabolic_diag = pd.merge(diagnosesList, metabolic, how = 'left', on = 'ICD9_CODE')
metabolic_diag = metabolic_diag[['SUBJECT_ID', 'metabolic disorder']]
metabolic_diag = metabolic_diag.groupby('SUBJECT_ID').agg(max)
master_disorders = pd.merge(master, metabolic_diag, how = 'left', on = 'SUBJECT_ID')
master_disorders = master_disorders[['SUBJECT_ID', 'metabolic disorder']]

##hypotension

hypotension = dict_diagnoses.copy()
hypotension['hypotension'] = 0
for i in hypotension.index:
    if has_string(str(hypotension['LONG_TITLE'][i]), 'hypotension'):
        hypotension['hypotension'][i] = 1
        
hypotension = pd.merge(diagnosesList, hypotension, how = 'left', on = 'ICD9_CODE')
hypotension = hypotension[['SUBJECT_ID', 'hypotension']]
master_disorders = pd.merge(master_disorders, hypotension, how = 'left', on = 'SUBJECT_ID')
master_disorders = master_disorders.groupby('SUBJECT_ID').agg(max)

##sepsis/septicemia

septicemia = dict_diagnoses.copy()
septicemia['septicemia'] = 0
for i in septicemia.index:
    if has_string(str(septicemia['LONG_TITLE'][i]), 'septicemia'):
        septicemia['septicemia'][i] = 1
        
septicemia = pd.merge(diagnosesList, septicemia, how = 'left', on = 'ICD9_CODE')
septicemia = septicemia[['SUBJECT_ID', 'septicemia']]
master_disorders = pd.merge(master_disorders, septicemia, how = 'left', on = 'SUBJECT_ID')
master_disorders = master_disorders.groupby('SUBJECT_ID').agg(max)

#sleep disorders

sleep = dict_diagnoses.copy()
sleep['sleep'] = 0
for i in sleep.index:
    if has_string(str(sleep['LONG_TITLE'][i]), 'sleep'):
        sleep['sleep'][i] = 1
        
sleep = pd.merge(diagnosesList, sleep, how = 'left', on = 'ICD9_CODE')
sleep = sleep[['SUBJECT_ID', 'sleep']]
master_disorders = pd.merge(master_disorders, sleep, how = 'left', on = 'SUBJECT_ID')
master_disorders = master_disorders.groupby('SUBJECT_ID').agg(max)

##anemia

anemia = dict_diagnoses.copy()
anemia['anemia'] = 0
for i in anemia.index:
    if has_string(str(anemia['LONG_TITLE'][i]), 'anemia'):
        anemia['anemia'][i] = 1
        
anemia = pd.merge(diagnosesList, anemia, how = 'left', on = 'ICD9_CODE')
anemia = anemia[['SUBJECT_ID', 'anemia']]
master_disorders = pd.merge(master_disorders, anemia, how = 'left', on = 'SUBJECT_ID')
master_disorders = master_disorders.groupby('SUBJECT_ID').agg(max)
master_disorders

#ion disorders

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [112]:
# lab = pd.merge(labEvents, dict_labItems, how = 'left', on = 'ITEMID')
# lab.to_csv('lab.csv', index = False)
# print(len(lab))
# lab = pd.merge(master, lab, how = 'left', on = 'SUBJECT_ID')
# print(len(lab))
master_disorders

,metabolic disorder,hypotension,septicemia,sleep,anemia
SUBJECT_ID,,,,,
11,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0
17,1.0,1.0,0.0,0.0,0.0
21,1.0,0.0,1.0,0.0,1.0
23,1.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0
32,1.0,0.0,0.0,1.0,0.0
36,0.0,0.0,0.0,0.0,0.0


In [118]:
###LABS

# dict_labItems = pd.read_csv('D_LABITEMS.csv')
# labEvents = pd.read_csv('LABEVENTS.csv')


##red blood cells - hemoglobin
hemoglobin
rbc = lab.copy()
rbc['rbc'] = 0
for i in rbc.index: 
    if has_string(str(rbc['LABEL'][i]), 'rbc') and rbc['FLAG'][i] == 'abnormal':
        rbc['rbc'][i] = 1

rbc = rbc[['SUBJECT_ID', 'rbc']]
master_labs = pd.merge(master, rbc, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##white blood cells
wbc = lab.copy()
wbc['wbc'] = 0
for i in wbc.index: 
    if has_string(str(wbc['LABEL'][i]), 'wbc') and wbc['FLAG'][i] == 'abnormal':
        wbc['wbc'][i] = 1

wbc = wbc[['SUBJECT_ID', 'wbc']]
master_labs = pd.merge(master_labs, wbc, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##bicarbonate
bicarbonate = lab.copy()
bicarbonate['bicarbonate'] = 0
for i in bicarbonate.index: 
    if has_string(str(bicarbonate['LABEL'][i]), 'bicarbonate') and bicarbonate['FLAG'][i] == 'abnormal':
        bicarbonate['bicarbonate'][i] = 1

bicarbonate = bicarbonate[['SUBJECT_ID', 'bicarbonate']]
master_labs = pd.merge(master_labs, bicarbonate, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##albumin
albumin = lab.copy()
albumin['albumin'] = 0
for i in albumin.index: 
    if has_string(str(albumin['LABEL'][i]), 'albumin') and albumin['FLAG'][i] == 'abnormal':
        albumin['albumin'][i] = 1

albumin = albumin[['SUBJECT_ID', 'albumin']]
master_labs = pd.merge(master_labs, albumin, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##chloride
chloride = lab.copy()
chloride['chloride'] = 0
for i in chloride.index: 
    if has_string(str(chloride['LABEL'][i]), 'chloride') and chloride['FLAG'][i] == 'abnormal':
        chloride['chloride'][i] = 1

chloride = chloride[['SUBJECT_ID', 'chloride']]
master_labs = pd.merge(master_labs, chloride, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##pco2
pco2 = lab.copy()
pco2['pco2'] = 0
for i in pco2.index: 
    if has_string(str(pco2['LABEL'][i]), 'pco2') and pco2['FLAG'][i] == 'abnormal':
        pco2['pco2'][i] = 1

pco2 = pco2[['SUBJECT_ID', 'pco2']]
master_labs = pd.merge(master_labs, pco2, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##potassium
potassium = lab.copy()
potassium['potassium'] = 0
for i in potassium.index: 
    if has_string(str(potassium['LABEL'][i]), 'potassium') and potassium['FLAG'][i] == 'abnormal':
        potassium['potassium'][i] = 1

potassium = potassium[['SUBJECT_ID', 'potassium']]
master_labs = pd.merge(master_labs, potassium, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##vitamin b12
b12 = lab.copy()
b12['b12'] = 0
for i in b12.index: 
    if has_string(str(b12['LABEL'][i]), 'b12') and b12['FLAG'][i] == 'abnormal':
        b12['b12'][i] = 1

b12 = b12[['SUBJECT_ID', 'potassib12um']]
master_labs = pd.merge(master_labs, b12, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##glucose
glucose = lab.copy()
glucose['glucose'] = 0
for i in glucose.index: 
    if has_string(str(glucose['LABEL'][i]), 'glucose') and glucose['FLAG'][i] == 'abnormal':
        glucose['glucose'][i] = 1

glucose = glucose[['SUBJECT_ID', 'glucose']]
master_labs = pd.merge(master_labs, glucose, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

##anion gap
anion = lab.copy()
anion['anion'] = 0
for i in anion.index: 
    if has_string(str(anion['LABEL'][i]), 'anion') and anion['FLAG'][i] == 'abnormal':
        anion['anion'][i] = 1

anion = anion[['SUBJECT_ID', 'glucose']]
master_labs = pd.merge(master_labs, anion, how = 'left', on = 'SUBJECT_ID')
master_labs = master_labs.groupby('SUBJECT_ID').agg(max)

master_labs

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

,rbc,wbc,bicarbonate,albumin,chloride,pco2
SUBJECT_ID,,,,,,
11,1.0,0,0,0,0,1
12,1.0,1,1,1,1,1
17,0.0,0,1,0,1,1
21,1.0,1,1,1,1,1
23,0.0,0,0,0,1,1
25,0.0,0,1,1,1,1
31,1.0,1,1,1,1,1
32,0.0,0,1,0,1,0
36,0.0,0,1,1,1,1


In [119]:
master_labs

,rbc,wbc,bicarbonate,albumin,chloride,pco2
SUBJECT_ID,,,,,,
11,1.0,0,0,0,0,1
12,1.0,1,1,1,1,1
17,0.0,0,1,0,1,1
21,1.0,1,1,1,1,1
23,0.0,0,0,0,1,1
25,0.0,0,1,1,1,1
31,1.0,1,1,1,1,1
32,0.0,0,1,0,1,0
36,0.0,0,1,1,1,1


In [81]:
###create final master file
master2 = master[['SUBJECT_ID', 'AGE', 'delirium']]
masterFinal = pd.merge(master_drugs, master_disorders, how = 'inner', on = 'SUBJECT_ID')
masterFinal = pd.merge(masterFinal, master_abuse, how = 'inner', on = 'SUBJECT_ID')
masterFinal = pd.merge(masterFinal, master_labs, how = 'inner', on = 'SUBJECT_ID')
masterFinal = pd.merge(masterFinal, master2, how = 'inner', on = 'SUBJECT_ID')
masterFinal = masterFinal.dropna()
masterFinal = masterFinal.drop_duplicates()
masterFinal.to_csv('masterFinal.csv', index = False) 
masterFinal = masterFinal[['SUBJECT_ID', 'AGE', 'benzodiazepine', 'opioid', 'anticholinergic', 'steroid', 'deep sedation', 'metabolic disorder', 'hypotension', 'septicemia', 'sleep', 'anemia', 'alcohol/drug abuse', 'rbc', 'wbc', 'bicarbonate', 'albumin', 'chloride', 'pco2', 'delirium']]
masterFinal

,SUBJECT_ID,benzodiazepine,opioid,anticholinergic,steroid,deep sedation,metabolic disorder,hypotension,septicemia,sleep,anemia,alcohol/drug abuse,AGE,delirium
0,11,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,1
1,12,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72,1
2,17,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,47,0
3,21,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,87,1
4,23,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,71,1
5,25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59,0
6,31,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72,1
7,36,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,70,1
8,37,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69,1
9,38,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,76,0


In [123]:
masterFinal1 = pd.merge(masterFinal, master_labs, how = 'inner', on = 'SUBJECT_ID')
masterFinal1
masterFinal = masterFinal1[['SUBJECT_ID', 'AGE', 'benzodiazepine', 'opioid', 'anticholinergic', 'steroid', 'deep sedation', 'metabolic disorder', 'hypotension', 'septicemia', 'sleep', 'anemia', 'alcohol/drug abuse', 'rbc', 'wbc', 'bicarbonate', 'albumin', 'chloride', 'pco2', 'delirium']]

In [125]:
masterFinal.to_csv('masterFinal.csv', index = False) 


In [49]:
keep = master[master['AGE'] < 150]
keep = keep[keep["AGE"] > 10]
master = keep

In [54]:
master.drop_duplicates(subset = 'SUBJECT_ID')

,SUBJECT_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,delirium,AGE
0,28,2177-09-01 07:15:00,2177-09-06 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,74
2,36,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,70
3,357,2197-12-06 07:13:00,2198-01-03 14:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,GASTROINTESTINAL BLEED,0,62
4,367,2160-09-08 11:47:00,2160-09-14 14:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,NECROTIZING FASCITITIS,1,60
5,369,2142-10-09 07:15:00,2142-10-16 14:20:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT; ? CA...,0,75
6,52,2191-01-10 02:12:00,2191-01-19 16:10:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SEIZURES,0,39
8,101,2196-09-26 18:36:00,2196-10-12 13:17:00,EMERGENCY,EMERGENCY ROOM ADMIT,RESPIRATORY FAILURE,0,82
9,106,2192-08-09 18:34:00,2192-08-15 21:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,CARDIAC ARREST,0,28
10,107,2115-02-20 17:41:00,2115-02-21 16:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,AV FISTULA;ANGIOPLASTY,0,63
12,109,2140-08-25 14:39:00,2140-09-02 18:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,HYPERTENSION EMERGENCY,1,20


,SUBJECT_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,delirium,AGE
0,23,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,71
1,25,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,ACUTE CORONARY SYNDROME,0,59
3,31,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,STATUS EPILEPTICUS,1,72
4,32,2170-04-04 08:00:00,2170-04-23 12:45:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,TRACHEAL STENOSIS/SDA,0,46
6,36,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,70
7,37,2183-08-21 16:48:00,2183-08-26 18:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,CONGESTIVE HEART FAILURE,1,69
8,38,2166-08-10 00:28:00,2166-09-04 11:30:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,76
9,41,2132-12-31 10:30:00,2133-01-27 15:45:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,RIGHT BRAIN STEM LESION/SDA,0,56
10,357,2198-08-02 04:49:00,2198-10-26 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,SEIZURE,1,62
11,359,2176-06-30 03:00:00,2176-07-03 15:12:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,R/O MYOCARDIAL INFARCTION,0,49
